In [1]:
import numpy as np
import random
from PIL import Image
from collections import deque
from keras.models import Sequential
from keras import initializers
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam, Nadam, Adamax
sizes = (80,80,4)

class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.99   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01  # exploration will not decay futher
        self.epsilon_decay = 0.00198
        self.learning_rate = 0.0001
        self.loss = 0
        self.model = self._build_model()
        self.weight_backup = 'model_weights.h5'
        self.old_I_2 = None
        self.old_I_3 = None
        self.old_I_4 = None
        self.old_I_1 = None

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', kernel_initializer='random_uniform', bias_initializer='zeros', padding='same', input_shape= sizes))#80*80*4
        model.add(Conv2D(64, kernel_size=4, subsample=(2, 2), activation='relu', padding='same'))
        model.add(Conv2D(64, kernel_size=3, subsample=(1, 1), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_size))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    
    def save_model(self):
            self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def stack(self, processed_observation):
        I_4 = self.old_I_3 if self.old_I_3 is not None else np.zeros((1, 80, 80))
        I_3 = self.old_I_2 if self.old_I_2 is not None else np.zeros((1, 80, 80))
        I_2 = self.old_I_1 if self.old_I_1 is not None else np.zeros((1, 80, 80))
        I_1 = processed_observation
        processed_stack = np.stack((I_4, I_3, I_2, I_1), axis = 3)
        #print(processed_stack.shape)
        self.old_I_4 = I_4
        self.old_I_3 = I_3
        self.old_I_2 = I_2
        self.old_I_1 = I_1
        return processed_stack

    def remember(self, state, action, reward, new_state, done):
        if len(self.memory) >= 50000:
            self.memory.popleft()
            self.memory.append([state, action, reward, new_state, done])
        else:
            self.memory.append([state, action, reward, new_state, done])    

    def memory_replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        Sample = random.sample(self.memory, batch_size)
        '''
        winsample = [s for s in self.memory if s[2]== 1.0]
        #print(winsample)
        tuple(winsample)
        if len(winsample) > 1:
            Samplewin = random.sample(winsample, 1)
            Sample += Samplewin
        else:  
            Sample += winsample
        '''
        inputs = np.zeros((len(Sample), state.shape[1], state.shape[2], state.shape[3])) # minibatch input
        targets = np.zeros((inputs.shape[0], self.action_size))
        
        for i in range(0, len(Sample)):
            sample_state = Sample[i][0]
            sample_action = Sample[i][1]
            sample_reward = Sample[i][2]
            sample_new_state = Sample[i][3]
            sample_done = Sample[i][4]
            
#             xxx = sample_new_state.reshape(sample_new_state.shape[1], sample_new_state.shape[2])
#             img = Image.fromarray(xxx, 'L')
#             img.show()
            
            inputs[i:i+1] = sample_state # slice of inputs setting = to state
            
            targets[i] = self.model.predict(sample_state)
            future_reward = self.model.predict(sample_new_state)
            
            if sample_done:
                targets[i, sample_action] = sample_reward
            #elif sample_reward == 1.0:
                #targets[i, sample_action] = sample_reward
            #elif sample_reward == -1.0:
                #targets[i, sample_action] = sample_reward
            else:
                targets[i, sample_action] = sample_reward + self.gamma * np.max(future_reward)
            print(sample_action, sample_reward, targets[i, sample_action])
        self.loss += self.model.train_on_batch(inputs, targets)
        print(self.loss)
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
    '''        
    def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation
        #stack4.append(processed_observation)
        #if len(stack4) == 4:
            #stack_of_observation = np.stack((processed_observation, processed_observation, processed_observation, processed_observation), axis=2)
            #stack_of_observation = stack_of_observation.reshape(stack_of_observation.shape[0], stack_of_observation.shape[1], stack_of_observation.shape[3], stack_of_observation.shape[2])
            #print(stack_of_observation.shape)
    '''

Using TensorFlow backend.


In [ ]:
import sys
import gym
from gym import wrappers
from scipy import misc
#import cv2


def RGBprocess(raw_img):
        processed_observation = Image.fromarray(raw_img, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1]) #1x80x80x1
        return processed_observation

'''
def RGBprocess(raw_img): 
        I = raw_img[35:195]
        I = I[::2, ::2, 0]
        I[I == 144] = 0
        I[I == 109] = 0
        I[I != 0] = 1
        processed_observation = I.astype(np.float32)
        return processed_observation

def RGBprocess(raw_img):
    grayscale_observation = raw_img.mean(2)
    resized_observation = misc.imresize(grayscale_observation, (80, 80)).astype(np.float32)
    processed_observation = resized_observation.reshape(1, resized_observation.shape[0], resized_observation.shape[1], 1)
    return processed_observation


def RGBprocess(raw_img):
    frame = cv2.cvtColor(cv2.resize(frame, (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, frame = cv2.threshold(frame, 1, 255, cv2.THRESH_BINARY)
    processed_observation = np.reshape(frame, (1, 84, 84, 1))
    return processed_observation
'''

batch_size = 32
#episodes = sys.argv[1] if len(sys.argv) > 1 else 5000
#env_name = sys.argv[2] if len(sys.argv) > 2 else "Pong-v0"

episodes = 2000
env_name = "Pong-v0"
D = 80*80

env = gym.make(env_name)

env = wrappers.Monitor(env, env_name, force=True)

agent = Agent(env.observation_space.shape, env.action_space.n)

for i_episodes in range(episodes):
    State = env.reset()
    state = RGBprocess(State)
    state = agent.stack(state)
    totalreward = 0
    num_actions = 0
    prev_x = None
    short_mem = []
    done = False
    while not done:
        if i_episodes % 20 == 0:
            env.render()
        action = agent.act(state)
        new_state, reward, done, info = env.step(action)
        new_state = RGBprocess(new_state)
        #new_state_dif = new_state - prev_x if prev_x is not None else np.zeros((1, 80, 80, 1))
        #prev_x = new_state
        new_state_dif = agent.stack(new_state)
        agent.remember(state, action, reward, new_state_dif, done)
        state = new_state_dif
        totalreward += reward
    agent.memory_replay(batch_size)
    if done:
        print("{} episode, score = {} ".format(i_episodes + 1, totalreward))
        agent.save_model()

env.close()
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

[2017-08-23 20:32:05,608] Making new env: Pong-v0
[2017-08-23 20:32:05,796] Clearing 6 monitor files from previous run (because force=True was provided)
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=8, activation="relu", kernel_initializer="random_uniform", bias_initializer="zeros", padding="same", input_shape=(80, 80, 4..., strides=(4, 4))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=4, activation="relu", padding="same", strides=(2, 2))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=3, activation="relu", padding="same", strides=(1, 1))`
[2017-08-23 20:32:05,886] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_

3 -1.0 11.5943750858
5 0.0 12.414872818
1 0.0 11.304931469
0 0.0 11.2278001499
1 0.0 11.8477407074
5 0.0 13.4171428299
5 0.0 12.1463279915
2 0.0 10.9319754124
1 0.0 13.1955414677
0 0.0 12.025244236
1 0.0 12.2590372467
4 0.0 12.1790442467
1 0.0 12.9953389835
1 0.0 13.2551580906
5 0.0 11.8549926281
3 0.0 13.1701394463
4 0.0 12.8318946075
2 0.0 12.7920973206
3 0.0 12.025244236
5 0.0 12.7192873192
2 0.0 10.9482598972
5 0.0 9.5664040947
2 0.0 12.1462165833
2 0.0 11.6406365776
5 0.0 12.0498730087
4 0.0 11.3560451889
2 0.0 13.2419864273
0 0.0 11.9560313988
4 0.0 10.8428053951
3 0.0 11.4592054367
0 0.0 10.8561772156
5 0.0 13.5895064735


[2017-08-23 20:32:27,073] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.21631.video000001.mp4


122.13835144
1 episode, score = -21.0 
1 0.0 32.3523921204
2 0.0 30.2706235313
1 0.0 32.021135788
1 0.0 31.7169724274
5 0.0 31.7236229324
5 0.0 30.0360034561
0 0.0 30.9439692307
2 0.0 29.8664797783
1 0.0 31.1812876511
2 0.0 32.7143518066
0 0.0 31.9195201492
5 0.0 32.5328356934
4 0.0 31.7900637817
5 0.0 31.1951343727
1 0.0 31.6273530006
3 0.0 31.2992160988
3 0.0 32.9155777359
3 0.0 31.6942791367
5 0.0 31.8925442505
1 0.0 31.6133098984
0 0.0 31.4858645439
4 0.0 33.5418732834
4 0.0 32.5708844376
2 0.0 31.0801591873
1 0.0 32.1205270386
5 0.0 29.0500254822
4 0.0 31.1599728012
4 0.0 30.5994232178
2 0.0 32.2376114273
5 0.0 29.322120266
3 0.0 31.8505754471
3 0.0 32.6370911407
200.278091431
2 episode, score = -21.0 
3 0.0 50.8283260345
4 0.0 50.5866910172
2 0.0 51.4951968384
3 0.0 49.9031240845
2 0.0 49.8714463806
5 0.0 51.7850848389
4 0.0 48.5199965286
5 0.0 49.8106401443
5 0.0 50.3691466141
1 0.0 50.7323072433
3 0.0 52.1771587372
1 0.0 49.9970771027
5 0.0 48.9310475922
4 0.0 49.9609846115
0 0

[2017-08-23 20:32:36,359] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.21631.video000008.mp4


...

4093049.0589
832 episode, score = -21.0 
4 0.0 2099.84317383
2 0.0 2103.60232178
2 0.0 2097.37905029
1 0.0 2100.79281006
4 0.0 2097.04502197
2 0.0 2093.95199707
0 0.0 2098.3980542
1 0.0 2102.65921143
4 0.0 2100.29611816
1 0.0 2096.07049072
4 0.0 2100.42035156
1 0.0 2102.12408936
1 0.0 2097.09553711
2 0.0 2093.91018311
3 0.0 2099.12581055
3 0.0 2097.71960449
1 0.0 2100.29587646
5 0.0 2099.25342773
2 0.0 2104.17659912
2 0.0 2100.97215088
0 0.0 2099.50552002
4 0.0 2100.04040039
1 0.0 2101.40068359
4 0.0 2104.85843262
4 0.0 2102.69619141
2 0.0 2100.87812988
0 0.0 2098.56555176
1 0.0 2102.23792969
4 0.0 2097.16442139
2 0.0 2098.19309326
2 0.0 2094.94344727
0 0.0 2099.99737793
4093080.79522
833 episode, score = -21.0 
0 0.0 2084.01477539
2 -1.0 2077.28408691
0 0.0 2082.62935547
2 0.0 2077.40067627
0 0.0 2084.19121582
2 0.0 2083.24182129
4 0.0 2085.32043457
4 0.0 2079.91531494
1 0.0 2085.77821289
4 0.0 2087.90395752
4 0.0 2081.22532471
2 0.0 2083.12677246
1 0.0 2089.65337646
1 0.0 2086.749602

5 0.0 1870.53780762
2 0.0 1871.64503174
4 0.0 1871.7046106
0 0.0 1871.67028931
5 0.0 1872.2909729
4 0.0 1872.26849487
4 0.0 1874.32027954
1 0.0 1874.6266333
4 0.0 1876.03356445
5 0.0 1876.33049194
4 0.0 1874.28160767
2 0.0 1871.40357422
5 0.0 1874.74434082
5 0.0 1873.64569702
5 0.0 1873.9767041
1 0.0 1872.14353638
1 0.0 1872.33327026
2 0.0 1874.26879761
0 0.0 1871.61929077
4 0.0 1873.28677368
5 0.0 1876.97752075
4 0.0 1876.22885742
5 0.0 1873.157948
4 0.0 1873.92123413
2 0.0 1875.17371948
2 0.0 1873.67542603
1 0.0 1872.28686401
4 0.0 1873.33765137
0 0.0 1874.38928467
2 0.0 1873.31650269
2 0.0 1872.04552734
1 0.0 1871.59898804
4093480.01522
845 episode, score = -21.0 
5 0.0 1855.0911731
4 0.0 1856.16032959
4 0.0 1851.27474243
2 0.0 1853.27154053
2 0.0 1855.37323608
5 0.0 1856.71152466
2 0.0 1856.64094849
5 0.0 1854.71122192
2 0.0 1854.377677
4 0.0 1854.08775879
4 0.0 1853.33305298
4 0.0 1852.02171387
2 0.0 1854.07772827
1 0.0 1851.88962524
2 0.0 1852.83164795
1 0.0 1854.68463501
5 0.0 1

4129084.34249
857 episode, score = -21.0 
2 0.0 1655.89887085
2 0.0 1656.50058105
1 0.0 1657.57130859
4 0.0 1653.41927856
1 0.0 1654.60300049
2 0.0 1653.91452026
5 0.0 1655.35492676
4 0.0 1653.9046106
1 0.0 1657.6307666
4 0.0 1654.5080127
4 0.0 1658.60058472
5 0.0 1656.42529175
4 0.0 1654.59019043
1 0.0 1660.47169922
2 0.0 1654.94947632
0 0.0 1654.80203979
4 0.0 1656.99823975
1 0.0 1659.18634277
2 0.0 1656.4707312
4 0.0 1653.26737061
4 0.0 1654.99008179
2 0.0 1654.66342529
4 0.0 1654.20540527
2 0.0 1657.79415527
4 0.0 1657.04694214
1 0.0 1656.96838989
4 0.0 1658.09978394
2 0.0 1655.97814819
2 0.0 1656.83388428
4 0.0 1659.46055054
1 0.0 1656.5321228
1 0.0 1654.76880615
4129134.87218
858 episode, score = -21.0 
0 0.0 1650.47284424
4 0.0 1654.32540894
2 0.0 1653.52381348
2 0.0 1656.1776709
2 0.0 1653.03062622
4 0.0 1652.69696045
5 0.0 1653.82049927
2 0.0 1655.1399353
1 0.0 1656.06890625
4 0.0 1654.94319214
4 0.0 1652.71327515
2 0.0 1653.32960815
5 0.0 1654.335802
5 0.0 1653.83814331
2 0.0

4130043.69425
870 episode, score = -21.0 
2 0.0 1738.46247803
1 0.0 1740.59293579
0 0.0 1743.78094849
2 0.0 1739.19410156
2 0.0 1737.37773193
0 0.0 1742.29522339
5 0.0 1740.10506592
1 0.0 1744.35039185
2 0.0 1742.15008301
1 0.0 1744.41504639
1 0.0 1743.58166748
2 0.0 1737.48988037
1 0.0 1758.29583252
4 0.0 1740.89095093
5 0.0 1740.7447229
2 0.0 1740.30011719
4 0.0 1741.07633423
5 0.0 1743.22951172
4 0.0 1743.10672852
0 0.0 1739.60825317
2 0.0 1742.56072998
2 0.0 1741.14219727
2 0.0 1740.49951904
1 0.0 1743.36667603
4 0.0 1742.23322754
0 0.0 1742.00192139
1 0.0 1743.422146
4 0.0 1742.30851685
2 0.0 1737.21917725
5 0.0 1743.22830322
2 0.0 1740.67112549
4 0.0 1738.63009644
4130057.7438
871 episode, score = -21.0 
1 0.0 1740.25346924
4 0.0 1739.3985791
1 0.0 1741.21047729
5 0.0 1739.54504883
2 0.0 1741.92699463
2 0.0 1741.74330322
4 0.0 1741.37483276
4 0.0 1740.97288696
1 0.0 1739.06128784
4 0.0 1741.3951355
1 0.0 1738.45401855
4 0.0 1739.85949951
4 0.0 1739.08364502
1 0.0 1742.33643311
4 

4130377.19964
883 episode, score = -21.0 
5 0.0 1748.1850708
2 0.0 1744.13805908
5 0.0 1745.20479858
4 0.0 1747.33078491
0 0.0 1748.36622437
0 0.0 1747.00473267
4 0.0 1747.07808838
2 0.0 1745.51236084
1 0.0 1749.40081787
2 0.0 1749.49665161
3 0.0 1747.16219971
2 0.0 1749.72203613
2 0.0 1747.90506226
4 0.0 1746.52991455
0 0.0 1748.32501465
1 0.0 1749.70608398
0 0.0 1748.50423462
5 0.0 1748.47571411
1 0.0 1748.85675293
0 0.0 1751.66759399
2 0.0 1746.18633911
2 0.0 1749.85255371
1 0.0 1747.47616699
5 0.0 1749.98887207
2 0.0 1747.83666138
1 0.0 1747.51060913
4 0.0 1749.19972412
5 0.0 1744.86158569
2 0.0 1749.39151245
2 0.0 1747.63858887
4 0.0 1748.52925049
4 0.0 1747.66614258
4130400.76436
884 episode, score = -21.0 
2 0.0 1741.46244873
2 0.0 1739.39084473
1 0.0 1741.35199219
2 0.0 1741.99334106
2 0.0 1739.56825195
4 0.0 1740.46543945
4 0.0 1738.07261719
1 0.0 1743.11313354
4 0.0 1737.8358728
0 0.0 1742.22984375
0 0.0 1741.47453369
5 0.0 1737.97170776
2 0.0 1740.54882568
0 0.0 1742.9218286

In [ ]:
import gym
from gym import wrappers
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

In [ ]:
processed_observation.shape

In [ ]:
reward

In [ ]:
test = agent.model.predict(state)

In [ ]:
test[0]

In [ ]:
# env.reset()
# env.close()

In [ ]:
len(agent.memory)

In [ ]:
processed_observation = Image.fromarray(state, 'RGB')
processed_observation = processed_observation.convert('L')
processed_observation = processed_observation.resize((80, 80))
processed_observation = np.array(processed_observation)
processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1)

In [ ]:
new_state

In [ ]:
def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation

In [ ]:
new_state_dif.shape

In [ ]:
new_state_dif = new_state_dif.reshape(new_state_dif.shape[1], new_state_dif.shape[2])
img = Image.fromarray(new_state_dif, 'L')

In [ ]:
#img = Image.fromarray(state, 'L')
img.show()

In [ ]:
thing = [1,2,3,4,5,6]
for x in thing[-2:]:
    print(x)

In [ ]:
        # way to adjust reward, think incorrect
        if reward == 0:
            num_actions += 1
            short_mem.append([state, action, reward, new_state_dif, done])
        else:
            num_actions += 1
            short_mem.append([state, action, reward, new_state_dif, done])
            if reward == -1.0:
                for m in short_mem:
                    m[2] = -1.0
                    agent.remember(m[0], m[1], m[2], m[3], m[4])
                num_actions = 0
            elif reward == 1.0:
                for m in short_mem:
                    m[2] = 1.0
                    agent.remember(m[0], m[1], m[2], m[3], m[4])
            short_mem = []

In [ ]:
        # early attempt at increasing samples with positive reward
        winsample = [s for s in self.memory if s[2]== 1.0]
        #print(winsample)
        tuple(winsample)
        if len(winsample) > 4:
            Samplewin = random.sample(winsample, 4)
            Sample += Samplewin
        else:  
            Sample += winsample